## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-17-22-45-29 +0000,startribune,Takeaways: Mats Zuccarello’s overtime goal lif...,https://www.startribune.com/takeaways-mats-zuc...
1,2026-01-17-22-44-56 +0000,nypost,Venezuelan acting president Delcy Rodriguez wa...,https://nypost.com/2026/01/17/world-news/venez...
2,2026-01-17-22-41-01 +0000,nyt,Trump Announces New Tariffs on Some European C...,https://www.nytimes.com/2026/01/17/us/politics...
3,2026-01-17-22-36-41 +0000,nypost,US strikes Al-Qaeda leader with direct ties to...,https://nypost.com/2026/01/17/world-news/us-ki...
4,2026-01-17-22-34-21 +0000,nyt,Greenlanders Protest Trump’s Takeover Plans,https://www.nytimes.com/2026/01/17/world/europ...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2360/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
20,trump,43
71,iran,12
258,ice,11
22,new,11
97,minneapolis,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
153,2026-01-17-04-00-00 +0000,wsj,"Entering the second year of his second term, P...",https://www.wsj.com/politics/policy/over-two-w...,85
73,2026-01-17-17-03-12 +0000,nypost,"Trump slaps increased tariffs on Denmark, Euro...",https://nypost.com/2026/01/17/world-news/how-t...,84
2,2026-01-17-22-41-01 +0000,nyt,Trump Announces New Tariffs on Some European C...,https://www.nytimes.com/2026/01/17/us/politics...,80
158,2026-01-17-03-06-57 +0000,cbc,Trump invites Carney to join 'Board of Peace' ...,https://www.cbc.ca/news/world/carney-invite-tr...,79
177,2026-01-17-01-17-25 +0000,nyt,Trump Announces Members for Gaza ‘Board of Pea...,https://www.nytimes.com/2026/01/16/us/politics...,78


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
153,85,2026-01-17-04-00-00 +0000,wsj,"Entering the second year of his second term, P...",https://www.wsj.com/politics/policy/over-two-w...
180,60,2026-01-17-00-59-04 +0000,cbc,U.S. Justice Department probing Minnesota Gov....,https://www.cbc.ca/news/world/tim-walz-jacob-f...
41,46,2026-01-17-20-14-57 +0000,bbc,Iran supreme leader acknowledges thousands kil...,https://www.bbc.com/news/articles/ckglee733wno...
67,40,2026-01-17-17-50-20 +0000,bbc,Blair and Rubio among names on senior executiv...,https://www.bbc.com/news/articles/c07xv92vrz2o...
2,37,2026-01-17-22-41-01 +0000,nyt,Trump Announces New Tariffs on Some European C...,https://www.nytimes.com/2026/01/17/us/politics...
21,35,2026-01-17-21-29-53 +0000,nypost,California’s largest hospital group owes patie...,https://nypost.com/2026/01/17/us-news/kaiser-p...
62,32,2026-01-17-18-01-16 +0000,nypost,Ex-Nickelodeon child star Kianna Underwood was...,https://nypost.com/2026/01/17/us-news/ex-nicke...
40,27,2026-01-17-20-19-00 +0000,nypost,"Lefty protesters allegedly stab Jan. 6 rioter,...",https://nypost.com/2026/01/17/us-news/minneapo...
37,26,2026-01-17-20-23-33 +0000,nypost,Tehran’s security forces may have used lethal ...,https://nypost.com/2026/01/17/world-news/tehra...
148,26,2026-01-17-05-44-12 +0000,nypost,CIA Director John Ratcliffe meets with Venezue...,https://nypost.com/2026/01/17/us-news/cia-dire...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
